In [3]:
# default_exp core

# to_timemap

> Exports a timemap-ready XLSX from some other data.

In [16]:
#export
import json
import xlsxwriter
from datetime import datetime
from fastscript import *

In [17]:
#export
DEFAULT_HEADERS = [
    "id",
    "description",
    "location",
    "date",
    "time",
    "latitude",
    "longitude",
    "category",
    "filter0",
    "colour"
]

In [18]:
#export
def dt(d: str) -> datetime:
    return datetime.strptime(d, '%Y-%m-%dT%H:%M:%SZ')

def getDate(item: str) -> datetime:
    lspt = item.get('lastSpotted')
    if lspt is not None:
        return dt(lspt)
    startTime = item.get('startTime')
    if startTime is not None:
        return dt(startTime)
    endTime = item.get('endTime')
    if endTime is not None:
        return dt(endTime)
    raise Exception("JSON value doesn't have lastSpotted, startTime, or endTime")
    
def from_fleming(jdata, color):
    headers = DEFAULT_HEADERS
    odata = []
    past_locations = {}
    for idx, d in enumerate(jdata):
        thedate = getDate(d)
        lat = d['coordinates']['lat']
        lon = d['coordinates']['lon']
        k = f"{lat}_{lon}"
        locname = k
        if past_locations.get(k) is not None:
            locname = past_locations[k]
        else:
            past_locations[f"{lat}_{lon}"] = locname
        rdata = [
            idx,
            d['targetId'],
            locname,
            datetime.strftime(thedate, '%m/%d/%Y'),
            datetime.strftime(thedate, '%H:%M:%S'),
            lat,
            lon,
            "alpha", # just one category
            d['targetId'],
            color
            
        ]
        odata.append(rdata)

    return headers, odata

In [19]:
#export
CONVERTER = from_fleming

def write_sheet(sheet, hdrs, rows):
    row = 0
    col = 0

    for hdr in hdrs:
        sheet.write(row, col, hdr)
        col += 1

    row = 1
    col = 0

    for rdata in rows:
        col = 0
        for cell in rdata:
            sheet.write(row, col, cell)
            col += 1
        row += 1
    
@call_parse
def write_xlsx_from_json(inp:Param("Input file", str),
                 outp:Param("Output file", str)="out.xlsx",
                 color:Param("The color of events", str)="red",
                 tabname:Param('Name of tab', str)="export_events"):
    workbook = xlsxwriter.Workbook(outp)
    worksheet = workbook.add_worksheet(tabname)

    with open(inp, 'r') as f:
        data = json.load(f)

    conv_data = CONVERTER(data, color)
    headers = conv_data[0]
    row_data = conv_data[1]

    write_sheet(worksheet, headers, row_data)
    
    workbook.close()

# Functions to generate .xlsx and configs files
Store the names of files processed in `created_sheets` for the next step.

In [30]:
#export
def random_distinct_colors(n):
    import random
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append(f"rgb({r},{g},{b}") 
    return ret

COLS = [
    "#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
        "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
        "#5A0007", "#809693", "#FEFFE6", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
        "#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
        "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
        "#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
        "#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
        "#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
        "#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
        "#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
        "#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
        "#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
        "#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C",
        "#83AB58", "#001C1E", "#D1F7CE", "#004B28", "#C8D0F6", "#A3A489", "#806C66", "#222800",
        "#BF5650", "#E83000", "#66796D", "#DA007C", "#FF1A59", "#8ADBB4", "#1E0200", "#5B4E51",
        "#C895C5", "#320033", "#FF6832", "#66E1D3", "#CFCDAC", "#D0AC94", "#7ED379", "#012C58"
]

def distinct_colors(n):
    return COLS[:n]

def derive_latlon(sheets, prefix):
    import pandas as pd
    # for the time being, just pick first
    sheet =  pd.read_excel(output_xlsx / f"{prefix}_{sheets[0]}.xlsx", sheet_name='export_events')
    for idx, row in sheet[['latitude', 'longitude']].iterrows():
        return f"[{row.latitude}, {row.longitude}]"

## setup output locations in global namespace...

In [21]:
#export
from typing import List
from pathlib import Path
import os
from string import Template

desktop = list(Path(os.getcwd()).parents)[-3] / 'Desktop'
targets = desktop/'TargetsIds'
countries = [Path(x) for x in targets.glob("**/*") if x.is_dir()]
output = desktop/'TargetsDatasheets'
output_xlsx = output/'datasheet-server'/'data'
output_xlsx.mkdir(parents=True, exist_ok=True)

In [22]:
#export
def get_files(country, heuristic='by_size') -> List:
    """
    Get files according to a country.
    TODO(lachlan): More heuristics for selecting releavnt. 
    """
    if country == 'bhr':
        w_folder = countries[0]
    elif country == 'rwa1':
        w_folder = countries[1]
    elif country == 'is1':
        w_folder = countries[2]
    elif country == 'rwa2':
        w_folder = countries[3]
    elif country == 'sau':
        w_folder = countries[4]
        
    files = []
    ctr = list(w_folder.glob("**/*"))
    if heuristic == 'by_size':
        ctr.sort(key=os.path.getsize, reverse=True)
        w_start = 0 if len(ctr) < 10 else int(len(ctr) / 2)
        w_end = 10 if len(ctr) < 10 else int(len(ctr) / 2) + 30
    else:
        w_start = 0
        w_end = len(ctr)
        
    for idx, x in enumerate(ctr):
        if not x.is_file(): continue
        if idx < w_start: continue
        if idx >= w_end: break
        files.append(x)
        
    return files

In [34]:
#export
def rewrite_xlsx_data(batch_prefix: str, heuristic='by_size') -> List:
    """
    Rewrite the XLSX data in datasheet-server/data.
    Returns a list of the sheets that are created, so that they can be threaded with 
    config creation.
    """
    created_sheets = []
    w_files = get_files(batch_prefix, heuristic)
    colors = distinct_colors(len(w_files))
    for idx, f in enumerate(w_files):
        # create a f"{f.name}.xlsx" in output using main()
        write_xlsx_from_json(f, output_xlsx/f"{batch_prefix}_{f.stem}.xlsx", colors[idx])
        created_sheets.append(f.stem)

    # make a metadata sheet for categories and filters
    workbook = xlsxwriter.Workbook(output_xlsx/"metadata.xlsx")
    write_sheet(workbook.add_worksheet('export_categories'),
                ['category', 'description'],
                [['alpha', 'the only category...']])
    write_sheet(workbook.add_worksheet('export_filters'),
                ['None'],
                [[x] for x in created_sheets])
    workbook.close()
    return created_sheets

In [31]:
#export
def gen_templates(created_sheets, w_prefix):
    """
    Generate templates from a list of `created_sheets`, and a `w_prefix` that distinguishes
    this set of targets from others.
    """
    # event_exts, category_ext, map_anchor
    tm_template = Template("""module.exports = {
      title: 'example',
      display_title: 'Target Trace',
      SERVER_ROOT: 'http://localhost:4040',
      EVENTS_EXT: [
          $event_exts
      ],
      CATEGORIES_EXT: '$categories_ext',
      SOURCES_EXT: '',
      NARRATIVES_EXT: '',
      FILTERS_EXT: '$filters_ext',
      SITES_EXT: '',
      DATE_FMT: 'MM/DD/YYYY',
      TIME_FMT: 'hh:mm:ss',

      store: {
        app: {
          map: {
            anchor: $map_anchor,
            maxZoom: 20,
            minZoom: 3,
            startZoom: 15
          },
          timeline: {
            range: [
              new Date('2020-03-01T00:00:00Z'),
              new Date('2020-05-01T00:00:00.000Z')
            ],
            rangeLimits: [
              new Date('2019-01-01T00:00:00.000Z'),
              new Date('2020-07-01T00:00:00.000Z')
            ],
            dimensions: {}
          }
        },
        features: {
          USE_COVER: false,
          USE_CATEGORIES: true,
          USE_FILTERS: true,
          USE_SITES: false,
          USE_SOURCES: false,
          USE_NARRATIVES: false,
          USE_SEARCH: false,
          FILTERS_AS_NARRATIVES: true
        }
      }
    }
    """)

    single_xlsx = Template("""{ name: '$name', path: 'data/$name.xlsx', tabs: timemap.default }""")

    dss_template = Template("""import { timemap } from './lib'

    export default {
      gsheets: [],
      xlsx: [
          { name: 'metadata', path: 'data/metadata.xlsx', tabs: timemap.default },
          $xlsx_sheets
      ]
    }
    """)

    dss_config = dss_template.substitute(
        xlsx_sheets=',\n'.join([single_xlsx.substitute(name=f"{w_prefix}_{s}") for s in created_sheets])
    )

    tm_config = tm_template.substitute(
        event_exts=", ".join([f"'/api/{w_prefix}_{x}/export_events/deeprows'" for x in created_sheets]),
        categories_ext="/api/metadata/export_categories/rows",
        filters_ext="/api/metadata/export_filters/tree",
        map_anchor=derive_latlon(created_sheets, w_prefix)
    )

    with (output/'timemap'/'config.js').open('w', encoding='utf-8') as f:
        f.write(tm_config)
    print("tm_config updated.")

    with (output/'datasheet-server'/'src'/'config.js').open('w', encoding='utf-8') as f:
        f.write(dss_config)
    print("dss_config updated.")

# Workflow to generate


In [36]:
prefix = 'bhr'
sheets = rewrite_xlsx_data(prefix, heuristic='by_size')
gen_templates(sheets, prefix)

tm_config updated.
dss_config updated.


In [33]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_statistics.ipynb.
Converted index.ipynb.
